In [3]:
from dotenv import load_dotenv
import os

load_dotenv() 

True

In [4]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [5]:
from datetime import datetime
import pandas as pd
from src.data_utils import load_and_process_taxi_data

from_year = 2024
# to_year = datetime.now().year
to_year = 2025
print(f"Download raw data from {from_year} to {to_year}")

rides = pd.DataFrame()
chunks = []
for year in range(from_year, to_year+1):

    rides_one_year = load_and_process_taxi_data(year)

    chunks.append(rides_one_year)
    break

# Concatenate all chunks at the end
rides = pd.concat(chunks, ignore_index=True)
print("Data loading complete.")

Download raw data from 2024 to 2025
File already exists for 2024-01.
Loading data for 2024-01...
Total records: 1,881,640
Valid records: 57,196
Records dropped: 1,824,444 (96.96%)
Successfully processed data for 2024-01.
File already exists for 2024-02.
Loading data for 2024-02...
Total records: 2,115,518
Valid records: 68,438
Records dropped: 2,047,080 (96.76%)
Successfully processed data for 2024-02.
File already exists for 2024-03.
Loading data for 2024-03...
Total records: 2,655,667
Valid records: 80,317
Records dropped: 2,575,350 (96.98%)
Successfully processed data for 2024-03.
File already exists for 2024-04.
Loading data for 2024-04...
Total records: 3,208,493
Valid records: 98,794
Records dropped: 3,109,699 (96.92%)
Successfully processed data for 2024-04.
File already exists for 2024-05.
Loading data for 2024-05...
Total records: 4,219,327
Valid records: 124,424
Records dropped: 4,094,903 (97.05%)
Successfully processed data for 2024-05.
File already exists for 2024-06.
Loadi

In [6]:
rides

,started_at,pickup_location_id
0,2024-01-22 12:18:53.512,5779
1,2024-01-13 18:52:26.496,5779
2,2024-01-31 16:59:37.612,5779
3,2024-01-04 07:17:28.970,5779
4,2024-01-12 09:06:28.532,5779
...,...,...
1348712,2024-12-15 15:05:05.351,5905
1348713,2024-12-30 09:08:31.156,5905
1348714,2024-12-30 16:25:25.701,5788
1348715,2024-12-30 16:47:39.666,5788


In [7]:
rides.shape

(1348717, 2)

In [8]:
from src.data_utils import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

In [9]:
ts_data.shape

(10985, 3)

In [10]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10985 entries, 0 to 10984
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   pickup_hour         10985 non-null  datetime64[ns]
 1   pickup_location_id  10985 non-null  int16         
 2   rides               10985 non-null  int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 128.9 KB


In [11]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2023-12-31 20:00:00,5626,4
1,2024-01-01 00:00:00,5626,42
2,2024-01-01 04:00:00,5626,9
3,2024-01-01 08:00:00,5626,40
4,2024-01-01 12:00:00,5626,76
...,...,...,...
10980,2024-12-31 04:00:00,6072,8
10981,2024-12-31 08:00:00,6072,80
10982,2024-12-31 12:00:00,6072,105
10983,2024-12-31 16:00:00,6072,63


In [12]:
import hopsworks

api_key = os.getenv('HOPSWORKS_API_KEY')  
project_name = os.getenv('HOPSWORKS_PROJECT_NAME')  

# pip install confluent-kafka
# Initialize connection to Hopsworks  
project = hopsworks.login(  
    api_key_value=api_key,  
    project=project_name  
)  
print(f"Successfully connected to Hopsworks project: {project_name}")

2025-05-11 05:32:58,927 INFO: Initializing external client
2025-05-11 05:32:58,928 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-11 05:32:59,983 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214695
Successfully connected to Hopsworks project: sp25_taxi_nmuppala


In [13]:
feature_store = project.get_feature_store()

In [14]:
FEATURE_GROUP_NAME = "citibike_hourly_features"
FEATURE_GROUP_VERSION = 1

In [15]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    primary_key=['pickup_location_id', 'pickup_hour'],
    event_time=['pickup_hour']
)

In [16]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1214695/fs/1202330/fg/1458598


Uploading Dataframe: 100.00% |██████████| Rows 10985/10985 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: citibike_hourly_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214695/jobs/named/citibike_hourly_features_1_offline_fg_materialization/executions


(Job('citibike_hourly_features_1_offline_fg_materialization', 'SPARK'), None)

In [39]:
from hsfs.feature_group import FeatureGroup

feature_store = project.get_feature_store()

# Create the feature group
feature_group = feature_store.create_feature_group(
    name="time_series_hourly_feature_group",
    version=1,
    description="Feature group for hourly time series data",
    primary_key=["timestamp"],  # Adjust primary key according to your dataset
    online_enabled=True
)
print("Feature group created successfully!")

Feature group created successfully!


In [40]:
feature_group = feature_store.create_feature_group(
    name="time_series_hourly_feature_group",
    version=1,
    description="Feature group of hourly time series data",  # Use the new 'id' column as the primary key
    online_enabled=True   # Enable online serving if needed
)
print("Feature group created with supported primary key for online serving.")

Feature group created with supported primary key for online serving.


In [41]:
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'])
ts_data['pickup_location_id'] = ts_data['pickup_location_id'].astype(int)
ts_data['rides'] = ts_data['rides'].astype(int)

In [42]:
FEATURE_GROUP_NAME = "time_series_hourly_feature_group"
FEATURE_GROUP_VERSION = 1

feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time series data at hourly frequency",
    primary_key=['pickup_location_id', 'pickup_hour'],
    event_time=['pickup_hour']
)

In [43]:
ts_data.rename(columns={
    "pickup_hour": 'pickup_hour',
    "pickup_location_id": 'pickup_location_id',
    "rides": 'rides'
}, inplace=True)

ts_data_selected = ts_data[['pickup_hour', 'pickup_location_id', 'rides']]

feature_group.insert(ts_data_selected, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1214695/fs/1202330/fg/1401999


Uploading Dataframe: 100.00% |██████████| Rows 2277600/2277600 | Elapsed Time: 00:55 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214695/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)

In [44]:
df_memory_mb = rides.memory_usage(deep=True).sum() / (1024 * 1024)  
print(f"DataFrame size: {df_memory_mb:.2f} MB")

DataFrame size: 857.47 MB


In [45]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2277600 entries, 0 to 2277599
Data columns (total 3 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pickup_hour         datetime64[ns]
 1   pickup_location_id  int64         
 2   rides               int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 52.1 MB


In [18]:
feature_store = project.get_feature_store()

In [19]:
FEATURE_GROUP_NAME = "time_series_hourly_feature_group"
FEATURE_GROUP_VERSION = 1

In [20]:
from hsfs.feature_group import FeatureGroup

# Create the feature group
feature_group = feature_store.create_feature_group(
    name="time_series_hourly_feature_group",
    version=1,
    description="Feature group for hourly time series data",
    primary_key=["timestamp"],  # Adjust primary key according to your dataset
    online_enabled=True
)
print("Feature group created successfully!")

Feature group created successfully!


In [21]:
feature_group = feature_store.create_feature_group(
    name="time_series_hourly_feature_group",
    version=1,
    description="Feature group for hourly time series data",  # Use the new 'id' column as the primary key
    online_enabled=True   # Enable online serving if needed
)
print("Feature group created with supported primary key for online serving.")

Feature group created with supported primary key for online serving.


In [22]:
ts_data.rename(columns={
    'pickup_hour': 'pickup_hour',
    'pickup_location_id': 'pickup_location_id',
    'rides': 'rides'
}, inplace=True)

In [23]:
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'])
ts_data['pickup_location_id'] = ts_data['pickup_location_id'].astype(int)
ts_data['rides'] = ts_data['rides'].astype(int)

In [24]:
FEATURE_GROUP_NAME = "time_series_hourly_feature_group"
FEATURE_GROUP_VERSION = 1

feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time series data at hourly frequency",
    primary_key=['pickup_location_id', 'pickup_hour'],
    event_time=['pickup_hour']
)

In [25]:
ts_data = ts_data.rename(columns={
    "pickup_hour": "timestamp",
    "rides": "ride_count",
    "pickup_location_id": "pickup_location"
})

In [26]:
# Rename the columns
ts_data.rename(columns={
    'timestamp': 'pickup_hour',
    'pickup_location': 'pickup_location_id',
    'ride_count': 'rides'
}, inplace=True)

# Select only the required columns
ts_data_selected = ts_data[['pickup_hour', 'pickup_location_id', 'rides']]

# Insert the selected columns into the feature group
feature_group.insert(ts_data_selected, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1214695/fs/1202330/fg/1401992


Uploading Dataframe: 100.00% |██████████| Rows 2277600/2277600 | Elapsed Time: 00:55 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214695/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)

In [27]:
df_memory_mb = rides.memory_usage(deep=True).sum() / (1024 * 1024)  
print(f"DataFrame size: {df_memory_mb:.2f} MB")

DataFrame size: 857.47 MB


In [28]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2277600 entries, 0 to 2277599
Data columns (total 3 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pickup_hour         datetime64[ns]
 1   pickup_location_id  int64         
 2   rides               int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 52.1 MB


In [ ]:
ts_data['pickup_hour_str'] = ts_data['pickup_hour'].dt.strftime('%Y-%m-%d %H:00:00')
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'])


ts_data.head

<bound method NDFrame.head of                 pickup_hour  pickup_location_id  rides      pickup_hour_str
0       2023-01-01 00:00:00                   2      0  2023-01-01 00:00:00
1       2023-01-01 01:00:00                   2      0  2023-01-01 01:00:00
2       2023-01-01 02:00:00                   2      0  2023-01-01 02:00:00
3       2023-01-01 03:00:00                   2      0  2023-01-01 03:00:00
4       2023-01-01 04:00:00                   2      0  2023-01-01 04:00:00
...                     ...                 ...    ...                  ...
2277595 2023-12-31 19:00:00                 263    188  2023-12-31 19:00:00
2277596 2023-12-31 20:00:00                 263    198  2023-12-31 20:00:00
2277597 2023-12-31 21:00:00                 263    232  2023-12-31 21:00:00
2277598 2023-12-31 22:00:00                 263    160  2023-12-31 22:00:00
2277599 2023-12-31 23:00:00                 263     95  2023-12-31 23:00:00

[2277600 rows x 4 columns]>

In [17]:

ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10985 entries, 0 to 10984
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   pickup_hour         10985 non-null  datetime64[ns]
 1   pickup_location_id  10985 non-null  int16         
 2   rides               10985 non-null  int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 128.9 KB


In [18]:
feature_group = feature_store.get_or_create_feature_group(
    name="citibike_hourly_features",
    version=1,
    description="Feature group for ride data",
    primary_key=["pickup_hour", "pickup_location_id"],
    event_time="pickup_hour",
    online_enabled=True
)


In [19]:
print(ts_data.dtypes)
print(ts_data.columns)

feature_group.schema


pickup_hour           datetime64[ns]
pickup_location_id             int16
rides                          int16
dtype: object
Index(['pickup_hour', 'pickup_location_id', 'rides'], dtype='object')


[Feature('pickup_hour', 'timestamp', None, True, False, None, None, 1458598),
 Feature('pickup_location_id', 'int', None, True, False, None, None, 1458598),
 Feature('rides', 'int', None, False, False, None, None, 1458598)]

In [20]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 10985/10985 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: citibike_hourly_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214695/jobs/named/citibike_hourly_features_1_offline_fg_materialization/executions


(Job('citibike_hourly_features_1_offline_fg_materialization', 'SPARK'), None)

In [21]:
df_memory_mb = rides.memory_usage(deep=True).sum() / (1024 * 1024)  
print(f"DataFrame size: {df_memory_mb:.2f} MB")

DataFrame size: 30.87 MB


In [22]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10985 entries, 0 to 10984
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   pickup_hour         10985 non-null  datetime64[ns]
 1   pickup_location_id  10985 non-null  int16         
 2   rides               10985 non-null  int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 128.9 KB
